In [17]:
# CARGAMOS LIBRERIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime

import boto3
import s3fs
import fastparquet
import awswrangler as wr
import os

import pyarrow.parquet as pq
import pyarrow as pa

import requests
import json
from io import BytesIO
from pandas.tseries.offsets import BDay

import math

In [19]:
# INICIALIZAMOS UNA SESION EN AWS
session = boto3.Session(
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
    region_name=os.environ['S3_REGION'])

s3_client = boto3.client('s3')

OBJETIVO DE ESTE NOTEBOOK:
- DESCARGAR DE DISTINTOS ORIGENES LOS DATOS DE LOS ACTIVOS DEL IBEX35
- COMPARAR LOS DATOS OBTENIDOS 

## CARGA DE COMPOSICIÓN HISTÓRICA DE IBEX35

In [20]:
# LEEMOS EL FICHERO CON LA COMPOSICIÓN HISTORICA
ibex_constituents = pd.read_excel('data_processing/Historical Composition IBEX35.xlsx')
ibex_constituents.head()

,Activo,Inclusion,Exclusion
0,ACE,1991-01-02,2003-06-02
1,ACX,1991-01-02,1991-07-01
2,ALB,1991-01-02,1991-07-01
3,ASL,1991-01-02,1994-01-03
4,BBV,1991-01-02,2000-01-31


In [21]:
# ibex_constituents['Exclusion'] -= BDay(1)

# ibex_constituents['Inclusion'] = ibex_constituents['Inclusion'].dt.date
# ibex_constituents['Exclusion'] = ibex_constituents['Exclusion'].dt.date

# OBTENEMOS EL DIA ACTUAL
today = datetime.date.today().strftime('%Y-%m-%d')

# COMPLETAMOS LOS DATOS NULOS EN EL FICHERO DE COMPOSICIÓN HISTORICA CON LA FECHA ACTUAL
ibex_constituents.fillna(today,inplace=True)

# DEFINIMOS UNA VARIABLE PARA TODOS LOS ACTIVOS
activos = ibex_constituents.Activo
ibex_constituents['Ticker'] = activos.apply(lambda x: x.split('_')[0])

unique_activos = list(set(activos))

### LEEMOS LOS DATOS DE UN FICHERO PICKLE

In [22]:
import pickle
with open('./Datos TFM/stock_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [23]:
# REALIZAMOS LA CARGA DE TODOS LOS DATOS HISTORICOS DEL ARCHIVO Y LOS CARGAMOS EN UN BUCKET DE S3
bucket_name = 'stock-market-historical-data'
raw_folder = 'raw/pickle_data/'
s3 = boto3.resource('s3')
failed_loads = []

for t in data.keys():
    try:
        file_name = t +'.parquet'
        full_path = raw_folder + file_name
        stock_data = data[t]
        table = pa.Table.from_pandas(stock_data)
        buf = BytesIO()
        pq.write_table(table, buf)

        s3.Object(bucket_name, full_path).put(Body=buf.getvalue())
    except:
        failed_loads.append(ticker)
        continue

In [24]:
close_series = {ticker: df.close
                for ticker, df in data.items()}
stock_df = pd.DataFrame(close_series)

In [25]:
# REALIZAMOS LA DESCARGA DE LOS DATOS DE YFINANCE Y LOS CARGAMOS EN UN BUCKET DE S3
raw_folder = 'processed/pickle_data/'
s3 = boto3.resource('s3')

file_name = t +'.parquet'
full_path = raw_folder + file_name
table = pa.Table.from_pandas(stock_df)
buf = BytesIO()
pq.write_table(table, buf)

s3.Object(bucket_name, full_path).put(Body=buf.getvalue())


{'ResponseMetadata': {'RequestId': 'A0X0GB9ZNDAGDV8H',
  'HostId': '738UkZbE0sPj/rtuCFkmv40UllvjPvuntvrDaJWw9u1d86D4amQA2TG3L0Fyw46ycp/BICsk+ug=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '738UkZbE0sPj/rtuCFkmv40UllvjPvuntvrDaJWw9u1d86D4amQA2TG3L0Fyw46ycp/BICsk+ug=',
   'x-amz-request-id': 'A0X0GB9ZNDAGDV8H',
   'date': 'Wed, 09 Aug 2023 13:17:33 GMT',
   'x-amz-version-id': 'xJmiJamA7jjAY8tRXzdZO7DsxvGUDIo_',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"9b2d56a45e6ba6152c8aa14196e3570d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"9b2d56a45e6ba6152c8aa14196e3570d"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'xJmiJamA7jjAY8tRXzdZO7DsxvGUDIo_'}

In [53]:
stock_df.columns

Index(['SAN', 'BTO_1', 'BKIA_0', 'BKIA', 'NTGY', 'TRE', 'ACX_0', 'ACX', 'FER',
       'ACS', 'ELE_0', 'ELE', 'SAB', 'CRI', 'OHL', 'IBR', 'EBRO_0', 'EBRO_1',
       'AMS', 'AENA', 'ITX', 'MTS', 'IAG', 'VIS_0', 'VIS', 'IBLA', 'BBVA',
       'MAP', 'BME', 'CIE', 'MAS', 'IBE', 'ABG', 'ENG', 'SCYR_1', 'REP', 'ABE',
       'ABG.P_0', 'ABG.P_1', 'ANA_0', 'ANA', 'EVA', 'GRF', 'CLNX', 'MRL',
       'BKT', 'POP', 'ENC', 'IDR', 'SGRE_0', 'SGRE', 'MEL', 'TL5', 'REE',
       'COL', 'JAZ', 'TEF', 'FCC', 'CABK', 'DIA', 'ALM', 'REE_0', 'CIN', 'LOR',
       'UNF', 'ZEL', 'NHH_0', 'NHH_1', 'MAP_0', 'ALT', 'BTO_0', 'FAD', 'DRC',
       'PRS_0', 'PRS_1', 'SCYR_0', 'TEM', 'CAR', 'AGS', 'COL_0', 'A3TV',
       'SGC'],
      dtype='object')

### LEEMOS LOS DATOS DE FICHEROS

In [29]:
ibex_file_data = pd.read_parquet('./Datos TFM/IBEX.parquet')
ibex_file_data = ibex_file_data.loc['2020-08-24':,:'VIS_0']
ibex_file_data

,ABE,ABG,ABG.P_0,ABG.P_1,ACS,ACX,ACX_0,AENA,ALM,AMS,...,SCYR_1,SGRE,SGRE_0,SLR,TEF,TL5,TRE,UNI,VIS,VIS_0
2020-08-24,NaN,NaN,NaN,NaN,18.598827,5.784362,NaN,122.00,9.059603,45.25,...,NaN,22.80,NaN,NaN,2.886744,NaN,NaN,NaN,61.697101,NaN
2020-08-25,NaN,NaN,NaN,NaN,18.092966,5.664651,NaN,122.50,8.988342,47.54,...,NaN,22.91,NaN,NaN,2.830378,NaN,NaN,NaN,61.162295,NaN
2020-08-26,NaN,NaN,NaN,NaN,17.974931,5.796000,NaN,124.20,8.874325,48.12,...,NaN,23.26,NaN,NaN,2.792532,NaN,NaN,NaN,61.599863,NaN
2020-08-27,NaN,NaN,NaN,NaN,17.915914,5.686265,NaN,124.20,8.793563,48.57,...,NaN,22.20,NaN,NaN,2.765959,NaN,NaN,NaN,61.259533,NaN
2020-08-28,NaN,NaN,NaN,NaN,17.890621,5.774386,NaN,128.80,8.812566,49.03,...,NaN,22.27,NaN,NaN,2.729724,NaN,NaN,NaN,60.384396,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-17,NaN,NaN,NaN,NaN,28.160000,8.982000,NaN,142.40,NaN,58.32,...,NaN,NaN,NaN,16.885,3.702000,NaN,NaN,0.9875,NaN,NaN
2023-03-20,NaN,NaN,NaN,NaN,28.570000,9.140000,NaN,141.95,NaN,58.26,...,NaN,NaN,NaN,16.620,3.754000,NaN,NaN,0.9965,NaN,NaN
2023-03-21,NaN,NaN,NaN,NaN,29.120000,9.320000,NaN,146.55,NaN,59.26,...,NaN,NaN,NaN,16.525,3.840000,NaN,NaN,1.0430,NaN,NaN
2023-03-22,NaN,NaN,NaN,NaN,28.990000,9.262000,NaN,145.50,NaN,58.74,...,NaN,NaN,NaN,16.255,3.832000,NaN,NaN,1.0200,NaN,NaN


In [99]:
len(stock_df.columns), len(ibex_file_data)

(82, 666)

In [37]:
sub = 'BTO'

for ticker in ibex_file_data.columns:
    if sub in ticker:
        print(ticker)

BTO_1


In [ ]:
ibex_file_data = ibex_file_data.rename(columns={
    'ACX_0': 'ACX_1',
    'BKIA_0': 'BKIA_1',
    'ABG.P_0':'ABG.P',
    'VIS_0': 'VIS_1',
    'EBRO_0': 'EBRO',
    'SGRE_0': 'SGRE_1',
    'ANA_0': 'ANA_1',
    })

In [45]:
for ticker in ibex_file_data.columns:
    print(ticker, ibex_file_data[ticker].dropna(axis=0).index)

ABE DatetimeIndex([], dtype='datetime64[ns]', freq=None)
ABG DatetimeIndex([], dtype='datetime64[ns]', freq=None)
ABG.P_0 DatetimeIndex([], dtype='datetime64[ns]', freq=None)
ABG.P_1 DatetimeIndex([], dtype='datetime64[ns]', freq=None)
ACS DatetimeIndex(['2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27',
               '2020-08-28', '2020-08-31', '2020-09-01', '2020-09-02',
               '2020-09-03', '2020-09-04',
               ...
               '2023-03-10', '2023-03-13', '2023-03-14', '2023-03-15',
               '2023-03-16', '2023-03-17', '2023-03-20', '2023-03-21',
               '2023-03-22', '2023-03-23'],
              dtype='datetime64[ns]', length=666, freq=None)
ACX DatetimeIndex(['2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27',
               '2020-08-28', '2020-08-31', '2020-09-01', '2020-09-02',
               '2020-09-03', '2020-09-04',
               ...
               '2023-03-10', '2023-03-13', '2023-03-14', '2023-03-15',
               '2023-03-16', '

In [55]:
complete_ibex_data = pd.concat([stock_df,ibex_file_data],axis=0)

In [115]:
ibex_file_data['EBRO_0'].dropna()

Series([], Name: EBRO_0, dtype: float64)

In [116]:
stock_df['EBRO_0'].dropna()

2010-07-26     9.682044
2010-07-27     9.619192
2010-07-28     9.626225
2010-07-29     9.528743
2010-07-30     9.347398
                ...    
2012-04-25    10.279672
2012-04-26    10.225900
2012-04-27    10.543810
2012-04-30    10.229907
2012-05-02    10.084172
Name: EBRO_0, Length: 455, dtype: float64

In [118]:
complete_ibex_data['EBRO_0'].isnull().sum()

4704

In [54]:
complete_ibex_data = pd.concat([stock_df,ibex_file_data],axis=0)

ibex_historical_data = {}
processed_folder = 'processed/bme-data/'
file_name = 'ibex_historical_data'
full_path = processed_folder + file_name

table = pa.Table.from_pandas(complete_ibex_data)
buf = BytesIO()
pq.write_table(table, buf)

s3.Object(bucket_name, full_path).put(Body=buf.getvalue())

{'ResponseMetadata': {'RequestId': 'GWAF9Q8M3PGR4JNF',
  'HostId': 'er9S5BAxx9g01MM+DybZxUWnm8mLmpvrny8IIjHWPzk3SqnDClN3Rlo+dwxZE0ZlnuqwiBlte9c=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'er9S5BAxx9g01MM+DybZxUWnm8mLmpvrny8IIjHWPzk3SqnDClN3Rlo+dwxZE0ZlnuqwiBlte9c=',
   'x-amz-request-id': 'GWAF9Q8M3PGR4JNF',
   'date': 'Wed, 09 Aug 2023 13:43:57 GMT',
   'x-amz-version-id': 'xnP16C0s1LFFUOKv5isSpvk5_SQGcg3K',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4ebe40679404e46526a452fe125045dc"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"4ebe40679404e46526a452fe125045dc"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'xnP16C0s1LFFUOKv5isSpvk5_SQGcg3K'}